<h1 style="color: rgba(237, 153, 29, 1);">Algoritmo de Detecção e Correção de Erros</h1>
<h2>Código de Hamming SEC-DED</h2>

<h3>Erros em Dados Armazenados</h3>

<p>Em qualquer sistema computacional, os dados armazenados na memória (RAM) ou em discos podem sofrer corrupção. Isso significa que um bit, ou uma sequência de bits, pode <strong>virar</strong> espontaneamente (0 vira 1, ou 1 vira 0) devido a diversas causas, como interferências eletromagnéticas, envelhecimento de componentes ou curtos-circuitos.</p>

<h3>Paridade Simples (RAID)</h3>

<p>Vimos que alguns níveis de RAID (como RAID 3, 4 ou 5) usam um <strong>bit de paridade</strong> (ou um disco de paridade) para garantir a tolerância a falhas. Essa abordagem é chamada de <strong>código de rasura (<i>erasure code</i>)</strong>.</p>

<p>Ele funciona perfeitamente no cenário do RAID porque nesse tipo de sistema sabemos qual disco falhou. Quando um disco para de responder, o sistema o identifica como <strong>apagado</strong> e usa a paridade para reconstruir os dados do disco.</p>

<h3>Corrupção de Bits</h3>

<p>Mas o que acontece se um único bit virar na memória RAM, ou em um disco que não falhou?</p> 

<p>Se usássemos apenas a paridade simples do RAID, o sistema faria a checagem e detectaria um erro de paridade. No entanto, ele não teria como saber onde o erro ocorreu. A paridade simples pode detectar um erro, mas não pode identificá-lo ou corrigi-lo sozinha.</p>

<p>É aqui que entra o <strong>Código de Hamming</strong>. Ele é um <strong>Código de Correção de Erro (ECC)</strong>. Ele adiciona redundância (bits de teste) que permitem não apenas a detecção de um erro, mas também a identificação da posição exata do bit que falhou, possibilitando sua correção.</p>

<h3>Hamming SEC-DED (13 bits)</h3>

<p>Nesta prática, vamos implementar o <strong>Hamming SEC-DED</strong> (<i>Single-Error-Correcting, Double-Error-Detecting</i>). Esta versão é capaz de corrigir um erro único e detectar a ocorrência de um erro duplo.</p>

<p>Em nossos testes, usaremos uma palavra de 13 bits. Essa palavra é composta por:</p>
<ul>
    <li><strong>8 bits de dados (D)</strong>: D<sub>1</sub> a D<sub>8</sub></li>
    <li><strong>4 bits de teste (T)</strong>: T<sub>1</sub>, T<sub>2</sub>, T<sub>4</sub>, T<sub>8</sub>. Estes ficam nas posições que são potências de 2 (1, 2, 4, 8).</li>
    <li><strong>1 bit de paridade geral (P)</strong>: Este bit extra permite a detecção de um erro duplo.</li>
</ul>

<p>A disposição dos bits na palavra (da posição 13 à 1) é a seguinte:</p>

<table style="width:100%; text-align:center; border: 1px solid #ccc;">
    <tr style="background-color:#f2f2f2;">
        <th>Posição</th>
        <th>13</th> <th>12</th> <th>11</th> <th>10</th> <th>9</th> <th>8</th> <th>7</th> <th>6</th> <th>5</th> <th>4</th> <th>3</th> <th>2</th> <th>1</th>
    </tr>
    <tr>
        <td><strong>Bit</strong></td>
        <td><strong>P</strong></td> <td>D<sub>8</sub></td> <td>D<sub>7</sub></td> <td>D<sub>6</sub></td> <td>D<sub>5</sub></td> <td><strong>T<sub>8<sub></strong></td> <td>D<sub>4</sub></td> <td>D<sub>3</sub></td> <td>D</sub>2<sub></td> <td><strong>T<sub>4</sub></strong></td> <td>D<sub>1</sub></td> <td><strong>T<sub>2</sub></strong></td> <td><strong>T<sub>1</sub></strong></td>
    </tr>
</table>

<h3>Verificação</h3>

<p>Considerando uma palavra de 13 bits (que pode ou não conter erros), a etapa de verificação de erros deve realizar o seguinte processo de validação:</p>

<ol>
    <li><strong>Ler a palavra:</strong> ler os 13 bits (P, D<sub>1</sub>-D<sub>8</sub>, T<sub>1</sub>-T<sub>8</sub>) da entrada.</li>
    <li><strong>Recalcular os bits de teste (T'):</strong> usando apenas os 8 bits de dados (B) lidos. </li>
    <li>Recalcular quais deveriam ser os valores dos bits de teste (T<sub>1</sub>', T<sub>2</sub>', T<sub>4</sub>', T<sub>8</sub>'). As regras de paridade (XOR) são:
        <ul>
            <li><code>T<sub>1</sub>' = (D<sub>1</sub> + D<sub>2</sub> + D<sub>4</sub> + D<sub>5</sub> + D<sub>7</sub>) % 2</code></li>
            <li><code>T<sub>2</sub>' = (D<sub>1</sub> + D<sub>3</sub> + D<sub>4</sub> + D<sub>6</sub> + D<sub>7</sub>) % 2</code></li>
            <li><code>D<sub>4</sub>' = (D<sub>2</sub> + D<sub>3</sub> + D<sub>4</sub> + D<sub>8</sub>) % 2</code></li>
            <li><code>T<sub>8</sub>' = (D<sub>5</sub> + D<sub>6</sub> + D<sub>7</sub> + D<sub>8</sub>) % 2</code></li>
        </ul>
    </li>
    <li><strong>Recalcular a paridade geral (P'):</strong> Recalcular a paridade geral (G') usando os 8 bits D e os 4 bits T'. O G' é o XOR de todos os outros 12 bits.
        <ul>
            <li><code>G' = (D<sub>1</sub> + D<sub>2</sub> + ... + D<sub>8</sub> + T<sub>1</sub>' + T<sub>2</sub>' + T<sub>4</sub>' + T<sub>8</sub>') % 2</code></li>
        </ul>
    </li>
    <li><strong>Calcular a posição do erro:</strong> A posição de erro é o resultado do XOR entre os bits T lidos e os bits T' recalculados.
        <ul>
            <li><code>S<sub>1</sub> = T<sub>1</sub> (lido) % 2 + T<sub>1</sub>' (recalculado) % 2</code> (Lembre-se: A + B % 2 é o mesmo que A XOR B para bits)</li>
            <li><code>S<sub>2</sub> = T<sub>2</sub> (lido) % 2 + T<sub>2</sub>' (recalculado) % 2</code></li>
            <li><code>S<sub>4</sub> = T<sub>4</sub> (lido) % 2 + T<sub>4</sub>' (recalculado) % 2</code></li>
            <li><code>S<sub>8</sub> = T<sub>8</sub> (lido) % 2 + T<sub>8</sub>' (recalculado) % 2</code></li>
        </ul>
        O número binário <code>S<sub>8</sub> S<sub>4</sub> S<sub>2</sub> S<sub>1</sub></code> forma a posição de erro (<code>erroPos</code>). Por exemplo, se <code>S<sub>8</sub>=1, S<sub>4</sub>=0, S<sub>2</sub>=1, S<sub>1</sub>=1</code>, o erro é 1011 (binário), que é 11 (decimal). Isso aponta que o bit na <strong>posição 11</strong> (D7) está errado.
    </li>
    <li><strong>Calcular o bit de erro geral:</strong> <code>erroPar = P (lido) % 2 + P' (recalculado) % 2</code>.</li>
    <li><strong>Diagnóstico (SEC-DED):</strong> Considerando o <code>erroPos</code> e o <code>erroPar</code>:
        <ul>
            <li><strong>Caso 1:</strong> Se <code>erroPos == 0</code> E <code>erroPar == 0</code>: A palavra está <strong>sem erros</strong>.</li>
            <li><strong>Caso 2:</strong> Se <code>erroPos > 0</code> E <code>erroPar == 1</code>: Há um <strong>erro único corrigível</strong>. A posição do erro é <code>erroPos</code>. Basta inverter o bit nessa posição para corrigi-lo.</li>
            <li><strong>Caso 3:</strong> Se <code>erroPos > 0</code> E <code>erroPar == 0</code>: Há um <strong>erro duplo (ou múltiplo)</strong> que não pode ser corrigido.</li>
            <li><strong>Caso 4:</strong> Se <code>erroPos == 0</code> E <code>erroPar == 1</code>: Há um erro único no bit P (posição 13). Os bits de dados estão corretos. Trate esse caso como um erro corrigível na posição 13.</li>
        </ul>
    </li>
</ol>